In [7]:
import numpy as np
import xarray as xr
import netCDF4 as nc

import pyart
import act

## Grab the CMAC DOD via ACT I/O

In [32]:
# Use ACT to query the ARM DOD API and initiate the CMAC DoD
dims = {'time' : 1, 'range' : 668, 'sweep' : 8, 'string_length' : 192}
dod = act.io.create_obj_from_arm_dod('xprecipradarcmacppi.c1', dims, version='2.0', fill_value=-9999)

In [33]:
dod

<xarray.Dataset>
Dimensions:                                   (time: 1, range: 668, sweep: 8,
                                               string_length: 192)
Coordinates:
  * time                                      (time) int64 0
  * range                                     (range) int64 0 1 2 ... 666 667
Dimensions without coordinates: sweep, string_length
Data variables: (12/48)
    azimuth                                   (time) int64 -9999
    elevation                                 (time) int64 -9999
    DBZ                                       (time, range) int64 -9999 ... -...
    VEL                                       (time, range) int64 -9999 ... -...
    WIDTH                                     (time, range) int64 -9999 ... -...
    ZDR                                       (time, range) int64 -9999 ... -...
    ...                                        ...
    time_coverage_end                         (string_length) int64 -9999 ......
    time_reference                            (string_length) int64 -9999 ......
    volume_number                             int64 -9999
    latitude                                  int64 -9999
    longitude                                 int64 -9999
    altitude                                  int64 -9999
Attributes: (12/25)
    command_line:          
    Conventions:           ARM-1.3 CF/Radial instrument_parameters
    process_version:       
    dod_version:           
    input_datastreams:     
    site_id:               
    ...                    ...
    translator:            https://www.arm.gov/capabilities/instruments/xprec...
    mentors:               https://www.arm.gov/connect-with-arm/organization/...
    source:                Colorado State University's X-Band Precipitation R...
    input_datastream:      xprecipradarS2.00
    field_names:           DBZ, VEL, WIDTH, ZDR, PHIDP, RHOHV, NCP, DBZhv, cb...
    history:

## Create the Encodings for NetCDF output

In [37]:
encodings = {
             'time'  : {'dtype' : np.double, '_FillValue' : -9999.0},
             'range' : {'dtype' : np.single, '_FillValue' : -9999.0},
             'azimuth' : {'dtype' : np.single, '_FillValue' : -9999.0},
             'elevation' : {'dtype' : np.single, '_FillValue' : -9999.0},
             'DBZ' : {'dtype' : np.double, '_FillValue' : -32768.0},
             'VEL' : {'dtype' : np.double, '_FillValue' : -32768.0},
             'WIDTH' : {'dtype' : np.double, '_FillValue' : -32768.0},
             'ZDR' : {'dtype' : np.double, '_FillValue' : -32768.0},
             'PHIDP' : {'dtype' : np.double, '_FillValue' : -32768.0},
             'RHOHV' : {'dtype' : np.double, '_FillValue' : -32768.0},
             'NCP' : {'dtype' : np.double, '_FillValue' : -32768.0},
             'DBZhv' : {'dtype' : np.double, '_FillValue' : -32768.0},
             'cbb_flag' : {'dtype' : np.double, '_FillValue' : -9999.0},
             'sounding_temperature' : {'dtype' : np.single, '_FillValue' : -9999},
             'height' : {'dtype' : np.single, '_FillValue' : -9999},
             'signal_to_noise_ratio' : {'dtype' : np.single, '_FillValue' : -32768.0},
             'velocity_texture' : {'dtype' : np.double, '_FillValue' : -9999}, 
             'gate_id' : {'dtype' : 'int16', '_FillValue' : -9999},
             'simulated_velocity' : {'dtype' : np.double, '_FillValue' : -32768.0},
             'corrected_velocity' : {'dtype' : np.double, '_FillValue' : -32768.0},
             'unfolded_differential_phase' : {'dtype' : np.double, '_FillValue' : -32768.0}, 
             'corrected_differential_phase' : {'dtype' : np.double, '_FillValue' : -32768.0},
             'filtered_corrected_differential_phase' : {'dtype' : np.double, '_FillValue' : -32768.0},
             'corrected_specific_diff_phase' : {'dtype' : np.double, '_FillValue' : -9999.0},
             'filtered_corrected_specific_diff_phase' : {'dtype' : np.double, '_FillValue' : -9999.0},
             'corrected_differential_reflectivity' : {'dtype' : np.double, '_FillValue' : 1e20},
             'corrected_reflectivity' : {'dtype' : np.double, '_FillValue' : 1e20},
             'height_over_iso0' : {'dtype' : np.single, '_FillValue' : -9999},
             'specific_attenuation' : {'dtype' : np.double, '_FillValue' : 1e20},
             'path_integrated_attenuation' : {'dtype' : np.double, '_FillValue' : 1e20},
             'specific_differential_attenuation' : {'dtype' : np.double, '_FillValue' : 1e20},
             'path_integrated_differential_attenuation' : {'dtype' : np.double, '_FillValue' : 1e20},
             'rain_rate_A' : {'dtype' : np.double, '_FillValue' : 1e20}, 
             'snow_rate_ws2012' : {'dtype' : np.double ,'_FillValue' : 1e20},
             'snow_rate_ws88diw' : {'dtype' : np.double, '_FillValue' : 1e20},
             'snow_rate_m2009_1' : {'dtype' : np.double, '_FillValue' : 1e20},
             'snow_rate_m2009_2' : {'dtype' : np.double, '_FillValue' : 1e20},
             'sweep_number' : {'dtype' : 'int16', '_FillValue' : -9999},
             'fixed_angle' : {'dtype' : np.single, '_FillValue' : -9999.0},
             'sweep_start_ray_index' : {'dtype' : 'int16', '_FillValue' : -9999},
             'sweep_end_ray_index' : {'dtype' : 'int16', '_FillValue' : -9999},
             'sweep_mode' : {'dtype' : np.str_},
             'nyquist_velocity' : {'dtype' : np.double, '_FillValue' : -9999.0},
             'time_coverage_start' : {'dtype' : np.str_}, 
             'time_coverage_end' : {'dtype' : np.str_},
             'time_reference' : {'dtype' : np.str_},
             'volume_number' : {'dtype' : 'int16', '_FillValue' : -9999},
             'latitude' : {'dtype' : np.double, '_FillValue' : -9999.0},
             'longitude' : {'dtype' : np.double, '_FillValue' : -9999.0},
             'altitude' : {'dtype' : np.double, '_FillValue' : -9999.0}
            }

In [92]:
joe = xr.Dataset()
mask = dod['DBZ'].where('time' + 'range' == -9999, 19)
joe['DBZ'] = mask.astype(np.double)

In [93]:
joe

<xarray.Dataset>
Dimensions:  (time: 1, range: 668)
Coordinates:
  * time     (time) int64 0
  * range    (range) int64 0 1 2 3 4 5 6 7 8 ... 660 661 662 663 664 665 666 667
Data variables:
    DBZ      (time, range) float64 19.0 19.0 19.0 19.0 ... 19.0 19.0 19.0 19.0

In [102]:
del joe['DBZ'].attrs['_FillValue']

In [103]:
joe

<xarray.Dataset>
Dimensions:  (time: 1, range: 668)
Coordinates:
  * time     (time) int64 0
  * range    (range) int64 0 1 2 3 4 5 6 7 8 ... 660 661 662 663 664 665 666 667
Data variables:
    DBZ      (time, range) float64 19.0 19.0 19.0 19.0 ... 19.0 19.0 19.0 19.0

## Modify the Datatype and Fill Values for the Initial DOD

In [112]:
# Create a blank DataSet to hold all the converted values
new_dod = xr.Dataset()
# Add the attributes in
new_dod.attrs = dod.attrs

In [113]:
new_dod

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes: (12/25)
    command_line:          
    Conventions:           ARM-1.3 CF/Radial instrument_parameters
    process_version:       
    dod_version:           
    input_datastreams:     
    site_id:               
    ...                    ...
    translator:            https://www.arm.gov/capabilities/instruments/xprec...
    mentors:               https://www.arm.gov/connect-with-arm/organization/...
    source:                Colorado State University's X-Band Precipitation R...
    input_datastream:      xprecipradarS2.00
    field_names:           DBZ, VEL, WIDTH, ZDR, PHIDP, RHOHV, NCP, DBZhv, cb...
    history:

In [114]:
# Loop over all the variables; update the FillValue and Data Type
for var in encodings:
    print(var, dod[var].dims)
    # Check if the DataArray is multidimensional (can't have 3 dimensions in this DOD)
    if len(dod[var].dims) == 2:
        # Character variables won't have a FillValue
        if encodings[var]['dtype'] != np.str_:
            # Convert all values within the DataArray to the correct Fill Value
            # NOTE: xr.where(condition, value_when_condition_is_not_met); so if every index is MVC, check for opposite
            mask = dod[var].where(dod[var].dims[0] + dod[var].dims[1] == -9999, encodings[var]['_FillValue'])
            # Append to the empty DataSet; make sure to change the data type
            new_dod[var] = mask.astype(encodings[var]['dtype'])
            # Delete _FillValue attribute; setting on write
            if '_FillValue' in new_dod[var].attrs:
                del new_dod[var].attrs['_FillValue']
        else:
            new_dod[var] = dod[var].astype(encodings[var]['dtype'])
    # most of the dimensions are one dimensional
    elif len(dod[var].dims) == 1:
        # Don't want repeating range values
        if var == 'range':
            new_dod[var] = dod[var].astype(encodings[var]['dtype'])
            if '_FillValue' in new_dod[var].attrs:
                del new_dod[var].attrs['_FillValue']
        else:
            # Character variables won't have a FillValue
            if encodings[var]['dtype'] != np.str_:
                # Convert all values within the DataArray to the correct Fill Value
                mask = dod[var].where(dod[var].dims[0] == -9999, encodings[var]['_FillValue'])
                # Append to the empty DataSet; make sure to change the data type
                new_dod[var] = mask.astype(encodings[var]['dtype'])
                if '_FillValue' in new_dod[var].attrs:
                    del new_dod[var].attrs['_FillValue']
            else:
                new_dod[var] = dod[var].astype(encodings[var]['dtype'])
    # For the three variables without dimensions
    else:
        new_dod[var] = dod[var].astype(encodings[var]['dtype'])
        if '_FillValue' in new_dod[var].attrs:
            del new_dod[var].attrs['_FillValue']

time ('time',)
range ('range',)
azimuth ('time',)
elevation ('time',)
DBZ ('time', 'range')
VEL ('time', 'range')
WIDTH ('time', 'range')
ZDR ('time', 'range')
PHIDP ('time', 'range')
RHOHV ('time', 'range')
NCP ('time', 'range')
DBZhv ('time', 'range')
cbb_flag ('time', 'range')
sounding_temperature ('time', 'range')
height ('time', 'range')
signal_to_noise_ratio ('time', 'range')
velocity_texture ('time', 'range')
gate_id ('time', 'range')
simulated_velocity ('time', 'range')
corrected_velocity ('time', 'range')
unfolded_differential_phase ('time', 'range')
corrected_differential_phase ('time', 'range')
filtered_corrected_differential_phase ('time', 'range')
corrected_specific_diff_phase ('time', 'range')
filtered_corrected_specific_diff_phase ('time', 'range')
corrected_differential_reflectivity ('time', 'range')
corrected_reflectivity ('time', 'range')
height_over_iso0 ('time', 'range')
specific_attenuation ('time', 'range')
path_integrated_attenuation ('time', 'range')
specific_di

In [115]:
new_dod

<xarray.Dataset>
Dimensions:                                   (time: 1, range: 668, sweep: 8,
                                               string_length: 192)
Coordinates:
  * time                                      (time) float64 -9.999e+03
  * range                                     (range) float32 0.0 1.0 ... 667.0
Dimensions without coordinates: sweep, string_length
Data variables: (12/48)
    azimuth                                   (time) float32 nan
    elevation                                 (time) float32 nan
    DBZ                                       (time, range) float64 nan ... nan
    VEL                                       (time, range) float64 nan ... nan
    WIDTH                                     (time, range) float64 nan ... nan
    ZDR                                       (time, range) float64 nan ... nan
    ...                                        ...
    time_coverage_end                         (string_length) <U21 '-9999' .....
    time_reference                            (string_length) <U21 '-9999' .....
    volume_number                             int16 -9999
    latitude                                  float64 -9.999e+03
    longitude                                 float64 -9.999e+03
    altitude                                  float64 -9.999e+03
Attributes: (12/25)
    command_line:          
    Conventions:           ARM-1.3 CF/Radial instrument_parameters
    process_version:       
    dod_version:           
    input_datastreams:     
    site_id:               
    ...                    ...
    translator:            https://www.arm.gov/capabilities/instruments/xprec...
    mentors:               https://www.arm.gov/connect-with-arm/organization/...
    source:                Colorado State University's X-Band Precipitation R...
    input_datastream:      xprecipradarS2.00
    field_names:           DBZ, VEL, WIDTH, ZDR, PHIDP, RHOHV, NCP, DBZhv, cb...
    history:

## Write the File

In [116]:
new_dod.to_netcdf('test_dod_v3.nc', encoding=encodings)

In [117]:
test = xr.open_dataset('test_dod_v3.nc', )

/opt/homebrew/Caskroom/miniforge/base/envs/sail-xprecip-dev/lib/python3.9/site-packages/xarray/conventions.py:523: SerializationWarning: variable 'sounding_temperature' has multiple fill values {-9999.0, '-9999'}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/opt/homebrew/Caskroom/miniforge/base/envs/sail-xprecip-dev/lib/python3.9/site-packages/xarray/conventions.py:523: SerializationWarning: variable 'height' has multiple fill values {-9999.0, '-9999'}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/opt/homebrew/Caskroom/miniforge/base/envs/sail-xprecip-dev/lib/python3.9/site-packages/xarray/conventions.py:523: SerializationWarning: variable 'velocity_texture' has multiple fill values {-9999.0, '-9999'}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [120]:
test.DBZ.data

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan

## Read the DOD via NetCDF4 to inspect how attributes, dtype, fill values are set

In [10]:
dod_v2 = nc.Dataset('/Users/jrobrien/ARM/data/CSU-XPrecipRadar/dod/cmac_dod_v2.nc')

In [11]:
dod_v2

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    command_line: 
    Conventions: ARM-1.3 CF/Radial instrument_parameters
    process_version: 
    dod_version: 
    input_datastreams: 
    site_id: 
    platform_id: 
    facility_id: 
    data_level: 
    location_description: 
    datastream: 
    institution: U.S. Department of Energy Atmospheric Radiation Measurement (ARM) Climate Research Facility
    references: See XPRECIPRADAR Instrument Handbook
    doi: 10.5439/1883164
    comment: This is highly experimental and initial data. There are many known and unknown issues. Please do not use before contacting the Translator responsible scollis@anl.gov
    attributions: This data is collected by the ARM Climate Research facility. Radar system is operated by the radar engineering team radar@arm.gov and the data is processed by the precipitation radar products team. LP code courtesy of Scott Giangrande BNL.
    vap_name: cmac
    known_issues: Fa